## DC Motor

In this tutorial we will model, identificy and control the speed of a DC motor.
We will cover the following topics

- System modelling
  - Representations
  - Simulation
  - saving/loading data
  
  
- System identification
  - Transfer function estimation
  - State space estimation


- Control design
  - Root locus
  - PID
  - Frequency domain (lead lag)
  - Pole placement

This tutorial is taken from
[Control Tutorials for Matlab and Simulink (CTMS)](http://ctms.engin.umich.edu/CTMS/index.php?example=MotorSpeed&section=SystemModeling)


In [ ]:
include("../PolynomialMatrices.jl/src/PolynomialMatrices.jl")
include("../SystemsBase.jl/src/SystemsBase.jl")
include("../ControlToolbox.jl/src/ControlToolbox.jl")
include("../IdentificationToolbox.jl/src/IdentificationToolbox.jl")

using PolynomialMatrices
using SystemsBase
using ControlToolbox
using IdentificationToolbox
using Plots
pyplot();

### System modelling

#### Physical setup

For this example, we will assume that the input of the system is the voltage source (V) applied to the motor's armature, while the output is the rotational speed of the shaft d(theta)/dt. The rotor and shaft are assumed to be rigid. We further assume a viscous friction model, that is, the friction torque is proportional to shaft angular velocity.


![title](figures/motor.png)

The physical constants are:

| parameter     | description                     |  value         |
| :------------ | :------------------------------ | :------------- |
| J             | moment of inertia of the rotor  | 0.01 kg.m^2    |
| b             | motor viscous friction constant | 0.1 N.m.s      |
| Ke            | electromotive force constant    | 0.01 V/rad/sec |
| Kt            | motor torque constant           | 0.01 N.m/Amp   |
| R             | electric resistance             | 1 Ohm          |
| L             | electric inductance             | 0.5 H          |


#### System equations

Motor torque is proportional to the armature current $i$ by a constant factor $Kt$
$$
T=K_{t}
$$
The back electromotive force, $e$, is proportional to the angular velocity of the shaft by a constant factor Ke.
$$
e = K_{e} \dot{\theta}
$$
In SI units, the motor torque and back electromotive force constant are equal, that is, $Kt = Ke$;
therefore, we will use $K$ to represent both the motor torque constant and the back electromotive force constant.

From the figure above, we can derive the following governing equations based on Newton's 2nd law
and Kirchhoff's voltage law.
$$
J\ddot{\theta} + b \dot{\theta} = K i
$$

$$ L \frac{di}{dt} + Ri = V - K\dot{\theta}$$

### System representations

#### Transfer function

Applying the Laplace transform, the above modeling equations can be expressed in terms of the Laplace variable s.

$$ s(Js + b)\Theta(s) = KI(s) $$

$$ (Ls + R)I(s) = V(s) - Ks\Theta(s) $$

We arrive at the following open-loop transfer function by eliminating I(s) between the two above equations, where the rotational speed is considered the output and the armature voltage is considered the input.

$$
P(s) = \frac {\dot{\Theta}(s)}{V(s)} = \frac{K}{(Js + b)(Ls + R) + K^2} \qquad \left[ \frac{rad/sec}{V} \right]
$$

In [ ]:
J = 0.01
b = 0.1
K = 0.01
R = 1.
L = 0.5
s = tf([1,0], 1)

Ptf = K/((J*s+b)*(L*s+R)+K^2)

#### State-Space

In state-space form, the governing equations above can be expressed by choosing the rotational speed and electric current as the state variables. 

$$
\frac{d}{dt}
\begin{bmatrix} \dot{\theta} \\ i \end{bmatrix}
    =
\begin{bmatrix}
    -\frac{b}{J} & \frac{K}{J} \\
    -\frac{K}{L} & -\frac{R}{L}
\end{bmatrix}
\begin{bmatrix} \dot{\theta} \\  i \end{bmatrix}  +
\begin{bmatrix} 0 \\ \frac{1}{L} \end{bmatrix} V
$$

$$
y =
\begin{bmatrix} 1 & 0 \end{bmatrix}
\begin{bmatrix} \dot{\theta} \\ i \end{bmatrix}
$$

In [ ]:
A = [-b/J   K/J
    -K/L   -R/L]
B = [0 1/L].'
C = [1. 0]
D = 0.
Pss = ss(A,B,C,D)

#### Design requirements

For a unit step command in motor speed, the control system's output should meet the following requirements.

- Settling time less than 2 seconds
- Overshoot less than 5%
- Steady-state error less than 1%

In [ ]:
plot(pzmap(Pss), aspect_ratio = :equal, ylim = (-1,1))

Because of the dominant pole, the open loop system behaves very much like the first order system

$$
\frac{0.1}{0.5s +1}
$$

In [ ]:
plot(step(Pss))
plot!(step(tf(0.1,[0.5, 1])))

### Simulation

We will now generate some data that can be used for identification

### System identification

#### load data

In [ ]:
data, header = readcsv("collected-data.csv", header=true)

u = data[:,2].'
y = data[:,3].'
zdata = iddata(y,u)

#### Transfer function identification

We identify a transfer function model of order 2
$$
    P(z) = \frac{b_0 z^{-1} + b_1 z^{-2}}{1 + f_1 z^{-1} + f_2 z^{-2}}
$$

We asssume that we know that the noise is white noise.
This is an Output-Error model.

In [ ]:
model   = OE(2,2,1);

Ptfinit = morsm(zdata, model)
Ptfid   = pem(zdata, Ptfinit, IdOptions(estimate_initial=false));

#### State-space identification

We identify a state-space model with 2 states

In [ ]:
Pssid = n4sid(zdata, 2)
Pssid = ss(Pssid)

#### Discretize true system

In [ ]:
Ts = 0.05
Pssd, xomap = c2d(Pss, Ts, Discretization.Bilinear()) # ZOH by default

discretizer = Discretization.ZOH()

Pssd, xomap = discretizer(Pss, Ts)
Ptfd        = discretizer(Ptf, Ts)

##### Compare time-responses

In [ ]:
plot(step(Pssd))
plot!(step(ss(Pssid))

### Root locus
We investigate the behaviour of a simple proportional controller in the feedback loop:

![title](figures/MotorSpeed_ControlRootLocus_feedback_motors.png)

where
$$
C(s) = c
$$

In [ ]:
rl = rootlocus(Ptf)
plot(rl)

In [ ]:
animate(rl, "animation.gif")

In [ ]:
plotly()
plot(rl)

The make the system as fast without too much overshoot we try to place the poles at $-2\pm 6j$

In [ ]:
c = 10
rl(c)

In [ ]:
Pcl = feedback(P*c,1)
step(Pcl)

In the above we saw that the overshoot and settling time criteria were met with the proportional controller,
but the steady-state error requirement was not.
A lag compensator is one type of controller known to be able to reduce steady-state error.

$$ C(s) = \frac {(s + 1)} { (s + 0.01) } $$

In [ ]:
C = tf([1, 1.], [1., .01])
rl = rootlocus(Ptf)
plot(rl)

We pick the gain 44.

In [ ]:
Pcl = feedback(44*P*C,1)
step(Pcl)

### PID

In this section we talk about PID

In [ ]:
pid(Code)

### Frequency domain

Here we will design a lag compensator using frequency domain methods.

In [ ]:
Pss(10)       # Pss(s = 10)
Pss(ω = 10)   # Pss(s = iω)
freqresp(10)  # Pss(s = iω)

In [ ]:
br = Bode(Pss)
plot(br)

A phase margin of 60 degrees is generally sufficient for stability margin. From the above Bode plot, this phase margin is achieved for a crossover frequency of approximately 10 rad/sec. The gain needed to raise the magnitude plot so that the gain crossover frequency occurs at 10 rad/sec appears to be approximately 40 dB.

In [ ]:
br(10)  

In [ ]:
C = 1/0.0139
br = Bode(C*Pss)
plot(br)

In [ ]:
Pcl = feedback(C*Pss,1)
plot(step(Pcl))

Note that the settling time is fast enough, but the overshoot and the steady-state error are too high. But can be fixed with a lag compensator.

#### Adding lag compensator
Consider the following lag compensator:

$$ C(s) = \frac {(s + 1)} { (s + 0.01) } $$

This lag compensator has a DC gain of 1/0.01 = 100 and will only add minor phase to around the crossover frequency of 10 rad/sec.

In [ ]:
C = 45*(s + 1)/(s + 0.01)
bode(C)

In [ ]:
Pcl = feedback(C*Ptf,1)
step(Pcl)

### Pole placement

Since both of the state variables in our problem are easy to measure (simply add an ammeter for current and a tachometer for the speed), we can design a full-state feedback controller for the system without worrying about having to add an observer. The control law for a full-state feedback system has the form 

$$ u = r - Kc x $$ 

and the associated schematic is shown below.

![title](figures/statefeedback_motors.png)

In [ ]:
sys_order = numstates(Pss)
sys_rank = rank(controllability(A,B))

We try to place the poles on the real axis. 

In [ ]:
Kc = place(A,B,[-6,-6])

In [ ]:
Pcl = ss(A-B*Kc,B,C,D)

In [ ]:
poles(Pcl)

In [ ]:
plot(step(Pcl))

To get the the correct steady state, we calculate the dcgain of the closed loop system and set the final control law to 

$$ u = \bar{N} r - Kc x $$ 

where $\bar{N}$ = 1/dcgain(Pcl).

In [ ]:
N̄ = 1/dcgain(Pcl)

In [ ]:
Pcl(0)

In [ ]:
step(N̄*Pcl)